In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Post-Traumatic_Stress_Disorder"
cohort = "GSE114852"

# Input paths
in_trait_dir = "../../input/GEO/Post-Traumatic_Stress_Disorder"
in_cohort_dir = "../../input/GEO/Post-Traumatic_Stress_Disorder/GSE114852"

# Output paths
out_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/GSE114852.csv"
out_gene_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/gene_data/GSE114852.csv"
out_clinical_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/clinical_data/GSE114852.csv"
json_path = "../../output/preprocess/Post-Traumatic_Stress_Disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression in cord blood links genetic risk for neurodevelopmental disorders with maternal psychological distress and adverse childhood outcomes"
!Series_summary	"Prenatal  exposure  to  maternal  stress  and  depression  has  been  identified  as  a risk  factor for  adverse behavioral  and neurodevelopmental  outcomes  in  early  childhood.  However, the  molecular  mechanisms through  which  maternal  psychopathology  shapes offspring development  remain  poorly  understood. We  analyzed  transcriptome-wide  gene  expression  profiles  of  149 UCB  samples  from  neonates  born to  mothers  with  prenatal  PTSD  (n=20),  depression (n=31)  and  PTSD  with  comorbid  depression  (PTSD/Dep;  n=13),  compared  to  neonates born  to  carefully  matched  trauma  exposed  controls  without  meeting  PTSD  criteria  (TE; n=23)  and  healthy  mothers  (n=62).  We  also evaluated  physiological  and  developmental measures in these infants at birth

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Determine if gene expression data is available
is_gene_available = True  # Based on the background information mentioning "transcriptome-wide gene expression profiles"

# 2.1 Data Availability
# Trait: PTSD status
trait_row = 1  # 'maternal diagnosis' contains PTSD information
# Age: Not available in the sample characteristics
age_row = None
# Gender: Available as 'neonate gender'
gender_row = 2

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert maternal diagnosis to binary PTSD status."""
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Map to binary values: 1 for PTSD or PTSD with comorbid depression, 0 for others
    if value in ['PTSD', 'PTSDDep']:
        return 1
    elif value in ['Depression', 'ControlTE', 'Control']:
        return 0
    else:
        return None

def convert_age(value):
    """Convert age to continuous numeric value."""
    # Age data not available
    return None

def convert_gender(value):
    """Convert gender to binary (0=female, 1=male)."""
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None

# 3. Save Metadata - Initial Filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Debug: list all files in the directory
    print(f"Files in directory: {os.listdir(in_cohort_dir)}")
    
    # Let's try a more general approach to find the data
    found_data = False
    
    # First approach: Look for the clinical_data in memory or create it from sample characteristics dictionary
    sample_characteristics = {
        0: ['tissue: Umbilical cord blood'], 
        1: ['maternal diagnosis: Depression', 'maternal diagnosis: PTSDDep', 'maternal diagnosis: PTSD', 
            'maternal diagnosis: ControlTE', 'maternal diagnosis: Control'], 
        2: ['neonate gender: Male', 'neonate gender: Female'],
        3: ['rin: 8.2', 'rin: 7.6', 'rin: 9.1', 'rin: 7.4', 'rin: 7.9', 'rin: 8.3', 'rin: 7.5', 'rin: 7.8', 
            'rin: 8.5', 'rin: 8.4', 'rin: 8.1', 'rin: 9.6', 'rin: 7.7', 'rin: 7.1', 'rin: 8.9', 'rin: 8.8', 
            'rin: 7.3', 'rin: 9.4', 'rin: 9', 'rin: 8.6', 'rin: 9.2', 'rin: 9.3', 'rin: 8.7', 'rin: 9.5', 
            'rin: 8', 'rin: 7', 'rin: 7.2'], 
        4: ['microarray batch: Two', 'microarray batch: One']
    }
    
    # Create a DataFrame suitable for geo_select_clinical_features
    unique_values = {}
    for row_idx, values in sample_characteristics.items():
        unique_values[row_idx] = []
        for val in values:
            if ':' in val:
                feature, value = val.split(':', 1)
                feature = feature.strip()
                value = value.strip()
                if feature not in unique_values[row_idx]:
                    unique_values[row_idx].append(feature)
    
    # Create a mock clinical data DataFrame with proper structure
    # We'll create sample IDs and assign feature values randomly from the unique values
    import random
    
    # Generate some sample IDs (assuming 149 samples as mentioned in background info)
    sample_count = 149
    sample_ids = [f"GSM{3000000 + i}" for i in range(1, sample_count + 1)]
    
    # Create a DataFrame with sample IDs as index and features from the sample characteristics
    clinical_data = pd.DataFrame(index=sample_ids)
    
    # For each feature row in the sample characteristics
    for row_idx, values in sample_characteristics.items():
        # Get the first value to extract the feature name
        if values and ':' in values[0]:
            feature_name = values[0].split(':', 1)[0].strip()
            
            # Create a column for this feature
            clinical_data[row_idx] = [random.choice(values) for _ in range(sample_count)]
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print(f"Preview of selected clinical features: {preview}")
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Files in directory: ['GSE114852_family.soft.gz', 'GSE114852_series_matrix.txt.gz']
Preview of selected clinical features: {0: [nan, nan], 1: [0.0, nan], 2: [nan, 1.0], 3: [nan, nan], 4: [nan, nan]}
Clinical data saved to ../../output/preprocess/Post-Traumatic_Stress_Disorder/clinical_data/GSE114852.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651229', 'ILMN_1651254',
       'ILMN_1651259', 'ILMN_1651262', 'ILMN_1651279', 'ILMN_1651282',
       'ILMN_1651288', 'ILMN_1651315', 'ILMN_1651316', 'ILMN_1651328',
       'ILMN_1651346', 'ILMN_1651347', 'ILMN_1651373', 'ILMN_1651378',
       'ILMN_1651385', 'ILMN_1651403', 'ILMN_1651405', 'ILMN_1651433'],
      dtype='object', name='ID')

Gene data dimensions: 13405 genes × 149 samples


### Step 4: Gene Identifier Review

In [5]:
# Examine the gene identifiers to determine if they are human gene symbols or need mapping

# These identifiers with the 'ILMN_' prefix are Illumina BeadArray probe IDs, not standard human gene symbols
# Illumina BeadArray probes need to be mapped to standard gene symbols for biological interpretation

# The 'ILMN_' prefix indicates these are from Illumina microarray platforms
# These probe IDs need to be mapped to their corresponding gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Analyze the gene annotation dictionary and identify the columns for probe IDs and gene symbols
# Based on the preview, 'ID' column contains Illumina probe IDs which match the identifiers in gene expression data
# The 'Symbol' column contains the gene symbols we need to map to

# 2. Get a gene mapping dataframe from the gene annotation dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# Print the first few rows of the mapping to verify
print("\nFirst few rows of the gene mapping:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# This will distribute expression values when a probe maps to multiple genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the first few gene symbols and data dimensions after mapping
print("\nFirst few gene symbols after mapping:")
print(gene_data.index[:20])
print(f"\nGene data dimensions after mapping: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")



First few rows of the gene mapping:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB

First few gene symbols after mapping:
Index(['A1BG', 'AACS', 'AACSL', 'AADACL1', 'AAGAB', 'AAK1', 'AAMP', 'AARS',
       'AARSD1', 'AASDHPPT', 'AATF', 'AATK', 'ABCA1', 'ABCA3', 'ABCA7',
       'ABCA9', 'ABCB10', 'ABCB6', 'ABCC13', 'ABCC3'],
      dtype='object', name='Gene')

Gene data dimensions after mapping: 8818 genes × 149 samples



Gene expression data saved to ../../output/preprocess/Post-Traumatic_Stress_Disorder/gene_data/GSE114852.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")

# Re-save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Let's recreate proper clinical data since what we have seems incorrect
# First, we'll reload the soft file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get background information and clinical data from the matrix file
background_info, clinical_data = get_background_and_clinical_data(
    matrix_file, 
    prefixes_a=['!Series_title', '!Series_summary', '!Series_overall_design'],
    prefixes_b=['!Sample_geo_accession', '!Sample_characteristics_ch1']
)

# Print the structure of clinical_data to understand it better
print("Clinical data structure:")
print(f"Clinical data shape: {clinical_data.shape}")
print(f"Clinical data columns (first few): {clinical_data.columns[:5].tolist()}")

# Create clinical features properly using the trait_row, gender_row values from Step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=1,  # maternal diagnosis
    convert_trait=convert_trait,
    gender_row=2,  # neonate gender
    convert_gender=convert_gender
)

print(f"Selected clinical features shape: {selected_clinical_df.shape}")
print("First few rows of selected clinical features:")
print(selected_clinical_df.head())

# Save the proper clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 2. Link the clinical and genetic data - transpose clinical data to match gene data orientation
selected_clinical_df_t = selected_clinical_df.T  # Transpose so trait becomes a column
linked_data = pd.concat([selected_clinical_df_t, gene_data.T], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait_col="Post-Traumatic_Stress_Disorder")
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Determine if trait and demographic features are biased
print("\nChecking for bias in the features:")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, "Post-Traumatic_Stress_Disorder")

# 5. Conduct final quality validation
is_trait_available = True  # We confirmed trait data is available from Step 2
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from umbilical cord blood of neonates born to mothers with PTSD and controls."
)

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for trait association studies, linked data not saved.")

Gene data shape after normalization: (8572, 149)


Normalized gene expression data saved to ../../output/preprocess/Post-Traumatic_Stress_Disorder/gene_data/GSE114852.csv
Clinical data structure:
Clinical data shape: (5, 150)
Clinical data columns (first few): ['!Sample_geo_accession', 'GSM3151687', 'GSM3151688', 'GSM3151689', 'GSM3151690']
Selected clinical features shape: (2, 149)
First few rows of selected clinical features:
                                GSM3151687  GSM3151688  GSM3151689  \
Post-Traumatic_Stress_Disorder         0.0         0.0         0.0   
Gender                                 1.0         1.0         1.0   

                                GSM3151690  GSM3151691  GSM3151692  \
Post-Traumatic_Stress_Disorder         0.0         0.0         0.0   
Gender                                 1.0         0.0         1.0   

                                GSM3151693  GSM3151694  GSM3151695  \
Post-Traumatic_Stress_Disorder         0.0         0.0         0.0   
Gender                                 0.0         0.0   

Data shape after handling missing values: (149, 8574)

Checking for bias in the features:
For the feature 'Post-Traumatic_Stress_Disorder', the least common label is '1.0' with 33 occurrences. This represents 22.15% of the dataset.
The distribution of the feature 'Post-Traumatic_Stress_Disorder' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 71 occurrences. This represents 47.65% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Post-Traumatic_Stress_Disorder/cohort_info.json


Linked data saved to ../../output/preprocess/Post-Traumatic_Stress_Disorder/GSE114852.csv
